<a href="https://www.kaggle.com/code/thiagohcarvalho/cria-o-e-an-lise-de-um-boletim-de-urna?scriptVersionId=135471378" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Esse projeto tem como objetivo analisar os registros de voto no segundo turno da eleição geral de 2022 para presidência do Brasil e transformar em um boletim de urna afim de comparar com a contagem feita pelo Tribunal Superior Eleitoral.

In [ ]:
!curl -s https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/notebooks/2022_11_29/rdv.py -o rdv.py

In [ ]:
!pip install asn1crypto==1.5.1

In [ ]:
!python rdv.py -r arquivoRDV.rdv.txt > rdv.txt

Será feita agora a limpeza dos dados para selecionar apenas as linhas do arquivo RDV referentes aos votos para presidente.



In [ ]:
content = []

with open(file="rdv.txt", mode="r") as fp:
  for line in fp.readlines():
    if "Governador" in line:
      break
    else:
      content.append(line)

In [ ]:
len(content)

In [ ]:
for line in content[:10]:
  print(line)

Faremos agora a extração dos dados

In [ ]:
import re

Criando um padrão para que apenas votos válidos sejam selecionados

In [ ]:
pattern = re.compile(pattern="\[(.*?)\]")

In [ ]:
votes = []

for line in content:

  if "branco" in line:

    votes.append({"voto": "branco", "quantidade": 1})

  if "nulo" in line:

    votes.append({"voto": "nulo", "quantidade": 1})

  if "nominal" in line:

    vote = re.findall(pattern=pattern, string=line)[0]
    votes.append({"voto": f"{vote}", "quantidade": 1})

In [ ]:
len(votes)

In [ ]:
for vote in votes[:10]:
  print(vote)

Será feita a agregação dos dados além da criação de uma nova coluna para visualizar os votos percentuais

In [ ]:
import pandas as pd

In [ ]:
votes_table = pd.DataFrame(votes)

In [ ]:
votes_table.tail(n=30)

In [ ]:
votes_table.shape

In [ ]:
votes_table.to_csv("rdv.csv", header=True, index=False)

In [ ]:
votes_table_agg = votes_table.groupby('voto').agg('sum').reset_index()

In [ ]:
votes_table_agg

In [ ]:
votes_table_agg = votes_table_agg.sort_values(by='quantidade', ascending=False)

In [ ]:
votes_table_agg

In [ ]:
votes_table_agg['quantidade_pct'] = round(100 * (votes_table_agg['quantidade'] / votes_table_agg['quantidade'].sum()), 2)

In [ ]:
votes_table_agg

# Visualização dos dados

In [ ]:
import seaborn as sns

In [ ]:
URNA = "Recife/PE - 7 - 276"

In [ ]:
x_column = 'voto'
y_column = 'quantidade'

In [ ]:
title = f'Apuração Presidente - Segundo turno de 2022 - Urna {URNA}'
x_label = 'Voto'
y_label = 'Quantidade'

In [ ]:
with sns.axes_style('whitegrid'):
  chart = sns.barplot(data=votes_table_agg, x=x_column, y=y_column)
  chart.set(title=title, xlabel=x_label, ylabel=y_label);

Nesta segunda visualização, vamos observar a **quantidade de votos relativos**.

In [ ]:
x_column = 'voto'
y_column = 'quantidade_pct'

In [ ]:
title = f'Apuração Presidente - Segundo turno de 2022 - Urna {URNA}'
x_lable = 'Voto'
y_lable = 'Quantidade (%)'

In [ ]:
with sns.axes_style('whitegrid'):
  chart = sns.barplot(data=votes_table_agg, x=x_column, y=y_column)
  chart.set(title=title, xlabel=x_label, ylabel=y_label);